In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%matplotlib notebook

import numpy as np
import matplotlib.pyplot as plt

In [120]:
from craco.preprocess import fast_preprpocess

In [121]:
from craft import uvfits, craco_plan
from craco import uvfits_meta, calibration

In [122]:
fname = "/data/craco/gup037/DATA/SB057841/DATA/DATA_01/craco/SB057841/scans/00/20240121205332/b00.uvfits"
meta_name = "/data/craco/gup037/DATA/SB057841/SB057841/SB57841.json.gz"
calname = "/data/craco/gup037/DATA/SB057841/SB057841/cal/00/b00.aver.4pol.smooth.npy"

In [123]:
values = craco_plan.get_parser().parse_args(["--flag-ants", "12,15,20,30", "--calibration", calname])

In [124]:
f = uvfits_meta.open(fname, metadata_file = meta_name)
f.set_flagants(values.flag_ants)
block0, uvws0 = next(f.fast_time_blocks(nt = 256))
block0.shape

(325, 1, 1, 1, 288, 1, 256)

In [125]:
plan = craco_plan.PipelinePlan(f, values)
calsoln = calibration.CalibrationSolution(plan)

/home/gup037/Codes/craft/src/craft/craco_plan.py:237: UserWarning: Cannot grid things on U=0 or V=0 blid=258.0 1-2 uvpix=(1, 0)
  warnings.warn(f'Cannot grid things on U=0 or V=0 blid={blid} {a1}-{a2} uvpix={uvpix}')
/home/gup037/Codes/craft/src/craft/craco_plan.py:237: UserWarning: Cannot grid things on U=0 or V=0 blid=258.0 1-2 uvpix=(2, 0)
  warnings.warn(f'Cannot grid things on U=0 or V=0 blid={blid} {a1}-{a2} uvpix={uvpix}')
/home/gup037/Codes/craft/src/craft/craco_plan.py:237: UserWarning: Cannot grid things on U=0 or V=0 blid=515.0 2-3 uvpix=(0, 255)
  warnings.warn(f'Cannot grid things on U=0 or V=0 blid={blid} {a1}-{a2} uvpix={uvpix}')
/home/gup037/Codes/craft/src/craft/craco_plan.py:237: UserWarning: Cannot grid things on U=0 or V=0 blid=526.0 2-14 uvpix=(0, 19)
  warnings.warn(f'Cannot grid things on U=0 or V=0 blid={blid} {a1}-{a2} uvpix={uvpix}')
/home/gup037/Codes/craft/src/craft/craco_plan.py:237: UserWarning: Cannot grid things on U=0 or V=0 blid=526.0 2-14 uvpix=(0, 20

In [126]:
input_block = block0.squeeze()[:, :, :block0.shape[-1]//1]
input_data = input_block.data
input_mask = input_block.mask

nbl, nf, nt = input_block.shape
isubblock = 0
output_buf = np.zeros_like(input_data)
#output_mask = np.zeros_like(input_mask)

Ai = np.zeros((nbl, nf), dtype=np.complex64)
Qi = np.zeros((nbl, nf), dtype=np.complex64)
N = np.ones((nbl, nf), dtype=np.int16)

cas = np.zeros((nf, nt), dtype=np.float64)
crs = np.zeros((nf, nt), dtype=np.float64)
cas_N = np.zeros((nf, nt), dtype=np.int16)

cal= calsoln.solarray.mean(axis=2).squeeze()
calsoln_data = cal.data
calsoln_mask = cal.mask

target_input_rms = None


In [127]:
#Computing cas and crs add 150ms of processing time per block (256 samples).
#Computing just the crs(aka pcb) adds 50ms
#Computing crs and cas adds 70ms (so 20ms extra from just crs/pcb)

In [128]:
cal.shape

(325, 288)

In [129]:
fast_preprpocess(input_block, input_mask, output_buf, isubblock, Ai, Qi, N, calsoln_data, calsoln_mask, cas, crs, cas_N, target_input_rms, reset_scales=True)

In [132]:
%timeit fast_preprpocess(input_block, input_mask, output_buf, isubblock, Ai, Qi, N, calsoln_data, calsoln_mask, cas, crs, cas_N, target_input_rms, reset_scales=True)

448 ms ± 2.77 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [92]:
Qi_old = Qi.copy()

In [93]:
N

array([[258, 258, 258, ..., 258, 258, 258],
       [258, 258, 258, ..., 258, 258, 258],
       [258, 258, 258, ..., 258, 258, 258],
       ...,
       [258, 258, 258, ..., 258, 258, 258],
       [258, 258, 258, ..., 258, 258, 258],
       [258, 258, 258, ..., 258, 258, 258]], dtype=int16)

In [32]:
type(input_block)

numpy.ma.core.MaskedArray

In [33]:
input_block.data

array([[[ 368.0417    -10.458313j ,  153.04167  -157.08333j  ,
          -30.125    +161.41669j  , ..., -119.166664  -19.833336j ,
         -163.875    -352.45834j  ,  353.6667    +29.916687j ],
        [-132.375    +196.95834j  , -359.83334  -156.79167j  ,
          182.66667  +226.625j    , ...,  606.5      +128.41667j  ,
          262.5      -247.62502j  ,   98.79167  +180.04167j  ],
        [  32.66667  +200.91667j  , -601.625    +303.5j      ,
         -386.62503  -228.75003j  , ..., -183.58333  -718.3334j   ,
         -383.        +68.04167j  ,  696.5      -393.83334j  ],
        ...,
        [-309.83334  -122.25j     , -326.83334  +355.83334j  ,
           60.791668 +174.95834j  , ..., -140.41669  -131.41667j  ,
         -231.58334   +35.875j    ,  215.25     -174.74998j  ],
        [-393.58334  +507.66666j  ,  607.0833   -110.33333j  ,
          -93.33334  -168.375j    , ...,    4.833336 +311.5417j   ,
           99.04167  +392.7917j   , -247.33336   -43.583332j ],
        [  7

In [34]:
output_buf.data

In [16]:
output_buf.shape

(325, 288, 8)